In [1]:
import re
import math
import itertools

In [2]:
data = open('data/day8.txt', 'r').read()

In [3]:
example1 = """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ"""

In [4]:
example2 = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""

In [5]:
example3 = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)"""

In [6]:
def parse_input(data):
    instructions, nodes = data.split('\n\n')
    network = {}
    for node in nodes.split('\n'):
        curr, left, right = re.findall("\w+", node)
        network[curr] = {'L': left, 'R': right}
    
    return instructions.strip(), network

In [7]:
def follow_instructions(instructions, network, source, sink):
    steps = 0
    curr = source
    while curr != sink:
        for direction in instructions:
            next_node = network[curr][direction]
            curr = next_node
            steps += 1
            if curr == sink:
                return steps 

In [8]:
def part1(data):
    instructions, network = parse_input(data)
    steps = follow_instructions(instructions, network, 'AAA', 'ZZZ')
    
    return steps
part1(data)

14429

In [9]:
def find_path(instructions, network, source, sink):
    steps = 0
    curr = source
    visits = {}
    graph_loops = {}
    loops = 0
    while curr != sink:
        for index, direction in enumerate(instructions):
            path = instructions * loops + instructions[:index]  # Get full path since beginning
            # Add/Update most recent visit's full path
            if curr not in visits:
                visits[curr] = path
            else:
                # Add path between visits to self to graph loops
                path_to_self = path.replace(visits[curr], '')                
                if curr not in graph_loops:
                    graph_loops[curr] = [path_to_self]
                else:
                    # Return None if we revisit the node with a prior loop path
                    if path_to_self in graph_loops[curr]:
                        return None
                    else:
                        graph_loops[curr].append(path_to_self)
                visits[curr] = path
            next_node = network[curr][direction]
            curr = next_node
            steps += 1
            # Return number of steps taken if we hit the sink
            if curr == sink:
                return steps
        loops += 1

In [10]:
def get_viable_path_lengths(instructions, network, source_nodes, sink_nodes):
    viable_path_lengths = {source: [] for source in source_nodes}
    for (source, sink) in list(itertools.product(source_nodes, sink_nodes)):
        path_length = find_path(instructions, network, source, sink)
        if path_length is not None:
            viable_path_lengths[source].append(path_length)
    
    return viable_path_lengths       
                   

In [ ]:
def part2(data):
    instructions, network = parse_input(data)
    source_nodes = [key for key in network.keys() if key[-1] == 'A']
    sink_nodes = [key for key in network.keys() if key[-1] == 'Z']
    viable_path_lengths = get_viable_path_lengths(instructions, network, source_nodes, sink_nodes)
    minimum_paths = [min(viable_path_lengths[source]) for source in viable_path_lengths]
    # The lcm of the minimum paths from each source node represents when they will first simultaneously reach a sink node
    lcm = 1
    for path_length in minimum_paths:
        lcm = math.lcm(lcm, path_length)

    return lcm
part2(data)